In [4]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from config import dbconfig

In [ ]:
#creating full_loans data set from database
engine = create_engine(dbconfig.aws_connection_string)
days = pd.read_sql_query("SELECT * FROM loan_dates", engine)
loans = pd.read_sql_query("SELECT * FROM loans", engine)
partners = pd.read_sql_query("SELECT * FROM partners", engine)
loans_days = pd.merge(loans, days, how = 'left', on = 'loan_id')
partners = partners.rename(columns = {"id": "partner_id"})
full_loans = pd.merge(loans_days, partners, how = "left", on = "partner_id", suffixes=("_loans", "_partners"))

In [ ]:
# creating a subset only with field partner loans
fl_subset = full_loans[full_loans.distribution_model == "field_partner"]

In [ ]:
#creating a new binary variable that is 0 if the loan was succesfully funded and 1 otherwise
fl_subset["status_loans_bi"] = np.where(fl_subset["status_loan"]=="funded", 0, 1)

In [ ]:
#Cleaning data set to select only funded loans and get rid of negative values in posted_to_raised_days
fl_subset_cl = fl_subset[fl_subset.status_loans_bi == 0]
fl_subset_cl = fl_subset_cl[fl_subset_cl.posted_to_raised_days >= 0]

In [ ]:
#Creating variables
fl_subset_cl['description_count'] = fl_subset_cl['description'].str.len()
fl_subset_cl["video_bi"] = np.where(fl_subset_cl["video_id"].isnull() == True, 0, 1)
fl_subset_cl["image_bi"] = np.where(fl_subset_cl["image_id"].isnull() == True, 0, 1)
fl_subset_cl["tags_bi"] = np.where(fl_subset_cl["tags"].isnull() == True, 0, 1)
fl_subset_cl["names_bi"] = np.where(fl_subset_cl["borrower_names"].isnull() == True, 0, 1)
fl_subset_cl["repayment_interval_bi"] = np.where(fl_subset_cl["repayment_interval"] == "monthly", 0, 1)

In [ ]:
#Creating variable for Gender (from Tom's code)
fl_subset_cl['num_borrowers_female'] = fl_subset_cl['borrower_genders'].str.count('female')
fl_subset_cl['num_borrowers_male'] = fl_subset_cl['borrower_genders'].str.count('^male') + fl_subset_cl['borrower_genders'].str.count('\, male')
fl_subset_cl['num_borrowers'] = fl_subset_cl['num_borrowers_female']+fl_subset_cl['num_borrowers_male']
fl_subset_cl['num_borrowers_female_pct'] = (fl_subset_cl['num_borrowers_female']*1.00)/fl_subset_cl['num_borrowers']